In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import torchvision.datasets as dsets


In [3]:
train_dataset=dsets.MNIST(root="./data",
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
test_dataset=dsets.MNIST(root="./data",
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 9912422/9912422 [00:04<00:00, 2121372.65it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 28881/28881 [00:00<00:00, 143570.95it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 1648877/1648877 [00:01<00:00, 1122943.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 4542/4542 [00:00<00:00, 1860039.91it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [11]:
train_dataset.__len__()

60000

In [12]:
len(test_dataset)

10000

In [13]:
batch_size=100
n_iters=3000
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True
                                         )
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [21]:
train_loader.__len__()
#600 batches

600

In [16]:
import torch
import torch.nn as nn

rnn=nn.RNN(input_size=10,hidden_size=20,num_layers=2)
input=torch.randn(5,3,10)
output,hidden=rnn(input)
output.shape,hidden.shape

(torch.Size([5, 3, 20]), torch.Size([2, 3, 20]))

In [17]:
#Code analogy to torch.nn.RNN
def rnn_step(input_seq,initial_hidden, input_linear,hidden_linear, activation):
    hidden_state=initial_hidden

    output_sequence=[]

    for x_t in input_seq:
        hidden_state=activation(input_linear(x_t) + hidden_linear(hidden_state))
        output_sequence.append(hidden_state)
    output_sequence=torch.stack(output_sequence,dim=0)

    return output_sequence,hidden_state

In [22]:
class RNN(nn.Module):
    #input_dim -> no. of features
    def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.layer_dim=layer_dim

        self.rnn=nn.RNN(input_size=input_dim,hidden_size=hidden_dim,num_layers=layer_dim,nonlinearity='relu',batch_first=True)

        self.fc=nn.Linear(in_features=hidden_dim,out_features=output_dim)
    
    def forward(self,x):

        #initial state
        h0=torch.zeros(self.layer_dim,x.size(0),self.hidden_dim) # multiple h0s for sequences in a batch

        out,hn=self.rnn(x, h0.detach())

        out=self.fc(out[:,-1,:])

        return out
        

        

In [23]:
input_dim=28
hidden_dim=100
layer_dim=1
output_dim=10

In [25]:
model = RNN(input_dim, hidden_dim, layer_dim, output_dim)

In [29]:
criterion = nn.CrossEntropyLoss()

In [26]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [27]:
len(list(model.parameters()))

6

In [28]:
# Input --> Hidden (A1)
list(model.parameters())[0].size()

torch.Size([100, 28])

In [30]:
# Number of steps to unroll
seq_dim = 28  
num_epochs=5
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        model.train()
       
        images = images.view(-1, seq_dim, input_dim).requires_grad_()

        
        optimizer.zero_grad()

        
      
        outputs = model(images)

         
        loss = criterion(outputs, labels)

        
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            model.eval()
           
            correct = 0
            total = 0
           
            for images, labels in test_loader:
                
                images = images.view(-1, seq_dim, input_dim)

               
                outputs = model(images)

             
                _, predicted = torch.max(outputs.data, 1)

               
                total += labels.size(0)

               
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

        
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.3016932010650635. Accuracy: 10.0600004196167
Iteration: 1000. Loss: 2.3038904666900635. Accuracy: 11.470000267028809
Iteration: 1500. Loss: 2.284355401992798. Accuracy: 15.420000076293945
Iteration: 2000. Loss: 2.2600982189178467. Accuracy: 18.93000030517578
Iteration: 2500. Loss: 1.8355488777160645. Accuracy: 31.8700008392334
Iteration: 3000. Loss: 1.2390812635421753. Accuracy: 60.560001373291016
